In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load data from ckextended.csv
data_df = pd.read_csv(r'D:\MTECH\PROJECT\CV\ORIGINAL\FINAL\computer-vision-hands-on\Project\Data\ckextended.csv')

# Select relevant columns
data_df = data_df[['emotion', 'pixels']]

# Convert pixels to numpy array
data_df['pixels'] = data_df['pixels'].apply(lambda x: np.array(x.split(), dtype='float32'))

# Define LBP parameters
n_points = 8
radius = 1

# Compute LBP features for all images
X = np.zeros((len(data_df), 256))
for i, pixels in enumerate(data_df['pixels']):
    pixels = pixels.reshape(48, 48)  # Reshape pixels into 2D image
    lbp = local_binary_pattern(pixels, n_points, radius)
    hist, _ = np.histogram(lbp, bins=np.arange(257), density=True)
    X[i] = hist

# Split data into train and test sets
y = data_df['emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train SVM classifier
svm = SVC(kernel='rbf', C=10, gamma=0.01)
svm.fit(X_train_pca, y_train)

# Predict labels for test set
y_pred = svm.predict(X_test_pca)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy}')


C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\skimage\feature\texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


Test accuracy: 0.6413043478260869


In [2]:
import joblib

# assuming "model" is the variable containing the trained model
filename = 'svm-ck+.joblib'
joblib.dump(svm, filename)


['svm-ck+.joblib']

In [3]:
import joblib

# assuming "model" is the variable containing the trained model
filename = 'pca-ck+.joblib'
joblib.dump(pca, filename)


['pca-ck+.joblib']

In [4]:
# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy}')

Test accuracy: 0.6413043478260869


In [9]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load data from ckextended.csv
data_df = pd.read_csv(r'D:\MTECH\PROJECT\CV\ORIGINAL\FINAL\computer-vision-hands-on\Project\Data\ckextended.csv')

# Select relevant columns
data_df = data_df[['emotion', 'pixels']]

# Convert pixels to numpy array
data_df['pixels'] = data_df['pixels'].apply(lambda x: np.array(x.split(), dtype='float32'))

# Define LBP parameters
n_points = 8
radius = 1

# Compute LBP features for all images
X = np.zeros((len(data_df), 256))
for i, pixels in enumerate(data_df['pixels']):
    pixels = pixels.reshape(48, 48)  # Reshape pixels into 2D image
    lbp = local_binary_pattern(pixels, n_points, radius)
    hist, _ = np.histogram(lbp, bins=np.arange(257), density=True)
    X[i] = hist

# Split data into train and test sets
y = data_df['emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Define the SVM classifier
svm = SVC()

# Define the grid of hyperparameters to search over
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.01, 0.1, 1, 20]
}

# Perform grid search to find the best hyperparameters
grid_search = GridSearchCV(svm, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_pca, y_train)
print(f'Best hyperparameters: {grid_search.best_params_}')

# Train SVM classifier with best hyperparameters
best_svm = SVC(kernel='rbf', C=grid_search.best_params_['C'], gamma=grid_search.best_params_['gamma'])
best_svm.fit(X_train_pca, y_train)

# Predict labels for test set
y_pred = best_svm.predict(X_test_pca)

# Calculate accuracy score and classification report
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy}')
print(classification_report(y_test, y_pred))


C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\skimage\feature\texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


Best hyperparameters: {'C': 100, 'gamma': 20}
Test accuracy: 0.7228260869565217
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.00      0.00      0.00        13
           2       0.50      0.20      0.29         5
           3       0.83      0.45      0.59        11
           4       0.00      0.00      0.00         6
           5       0.92      0.67      0.77        18
           6       0.72      0.97      0.83       118
           7       0.00      0.00      0.00         4

    accuracy                           0.72       184
   macro avg       0.37      0.29      0.31       184
weighted avg       0.61      0.72      0.65       184



C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344

In [12]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load data from ckextended.csv
data_df = pd.read_csv(r'D:\MTECH\PROJECT\CV\ORIGINAL\FINAL\computer-vision-hands-on\Project\Data\ckextended.csv')

# Select relevant columns
data_df = data_df[['emotion', 'pixels']]

# Convert pixels to numpy array
data_df['pixels'] = data_df['pixels'].apply(lambda x: np.array(x.split(), dtype='float32'))

# Define LBP parameters
n_points = 8
radius = 1

# Compute LBP features for all images
X = np.zeros((len(data_df), 256))
for i, pixels in enumerate(data_df['pixels']):
    pixels = pixels.reshape(48, 48)  # Reshape pixels into 2D image
    lbp = local_binary_pattern(pixels, n_points, radius)
    hist, _ = np.histogram(lbp, bins=np.arange(257), density=True)
    X[i] = hist

# Split data into train and test sets
y = data_df['emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Define the SVM classifier
svm = SVC()

# Define the grid of hyperparameters to search over
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.01, 0.1, 1, 20]
}

# Perform nested cross-validation to find the best hyperparameters
nested_scores = cross_val_score(GridSearchCV(svm, param_grid, cv=5, n_jobs=-1), X=X_train_pca, y=y_train, cv=5, n_jobs=-1)
best_score = nested_scores.mean()

print(f'Best cross-validation score: {best_score:.2f}')

# Train SVM classifier with best hyperparameters on the full training set
# Train SVM classifier with best hyperparameters on the full training set
best_svm_full = SVC(kernel='rbf', C=grid_search.best_params_['C'], gamma=grid_search.best_params_['gamma'])
best_svm_full.fit(X, y)


C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\skimage\feature\texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


Best cross-validation score: 0.67


SVC(C=100, gamma=20)

In [13]:
# Predict labels for test set
y_pred = best_svm.predict(X_test_pca)

# Calculate accuracy score and classification report
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy}')
print(classification_report(y_test, y_pred))


Test accuracy: 0.7119565217391305
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.00      0.00      0.00        13
           2       0.33      0.20      0.25         5
           3       0.67      0.36      0.47        11
           4       0.00      0.00      0.00         6
           5       0.92      0.67      0.77        18
           6       0.72      0.97      0.82       118
           7       0.00      0.00      0.00         4

    accuracy                           0.71       184
   macro avg       0.33      0.27      0.29       184
weighted avg       0.60      0.71      0.64       184



C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344

In [15]:
from sklearn.metrics import confusion_matrix

# Predict labels for test set
y_pred = best_svm.predict(X_test_pca)

# Calculate accuracy score, classification report, and confusion matrix
accuracy = accuracy_score(y_test, y_pred)
print(f'Test accuracy: {accuracy}')
print(classification_report(y_test, y_pred))
conf_mat = confusion_matrix(y_test, y_pred)
print(f'Confusion matrix:\n{conf_mat}')


Test accuracy: 0.7119565217391305
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.00      0.00      0.00        13
           2       0.33      0.20      0.25         5
           3       0.67      0.36      0.47        11
           4       0.00      0.00      0.00         6
           5       0.92      0.67      0.77        18
           6       0.72      0.97      0.82       118
           7       0.00      0.00      0.00         4

    accuracy                           0.71       184
   macro avg       0.33      0.27      0.29       184
weighted avg       0.60      0.71      0.64       184

Confusion matrix:
[[  0   0   1   0   0   0   8   0]
 [  0   0   0   0   0   0  13   0]
 [  1   0   1   0   0   0   3   0]
 [  0   0   0   4   0   0   7   0]
 [  0   1   0   0   0   0   5   0]
 [  0   0   1   0   0  12   5   0]
 [  0   0   0   2   1   1 114   0]
 [  0   0   0   0   0   0   4   0]]


C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rahul\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344